In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -20.000000. running mean: -20.000000
episode 2.000000, reward total was -21.000000. running mean: -20.010000
episode 3.000000, reward total was -21.000000. running mean: -20.019900
episode 4.000000, reward total was -20.000000. running mean: -20.019701
episode 5.000000, reward total was -20.000000. running mean: -20.019504
episode 6.000000, reward total was -21.000000. running mean: -20.029309
episode 7.000000, reward total was -21.000000. running mean: -20.039016
episode 8.000000, reward total was -21.000000. running mean: -20.048626
episode 9.000000, reward total was -21.000000. running mean: -20.058139
episode 10.000000, reward total was -21.000000. running mean: -20.067558
episode 11.000000, reward total was -17.000000. running mean: -20.036882
episode 12.000000, reward total was -21.000000. running mean: -20.046514
episode 13.000000, reward total was -21.000000. running mean: -20.056049
episode 14.000000, reward total was -20.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.306171
episode 115.000000, reward total was -21.000000. running mean: -20.313109
episode 116.000000, reward total was -21.000000. running mean: -20.319978
episode 117.000000, reward total was -19.000000. running mean: -20.306778
episode 118.000000, reward total was -20.000000. running mean: -20.303710
episode 119.000000, reward total was -21.000000. running mean: -20.310673
episode 120.000000, reward total was -20.000000. running mean: -20.307567
episode 121.000000, reward total was -21.000000. running mean: -20.314491
episode 122.000000, reward total was -19.000000. running mean: -20.301346
episode 123.000000, reward total was -20.000000. running mean: -20.298332
episode 124.000000, reward total was -21.000000. running mean: -20.305349
episode 125.000000, reward total was -19.000000. running mean: -20.292296
episode 126.000000, reward total was -20.000000. running mean: -20.289373
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.414238
episode 226.000000, reward total was -21.000000. running mean: -20.420096
episode 227.000000, reward total was -21.000000. running mean: -20.425895
episode 228.000000, reward total was -21.000000. running mean: -20.431636
episode 229.000000, reward total was -20.000000. running mean: -20.427319
episode 230.000000, reward total was -21.000000. running mean: -20.433046
episode 231.000000, reward total was -21.000000. running mean: -20.438716
episode 232.000000, reward total was -21.000000. running mean: -20.444329
episode 233.000000, reward total was -21.000000. running mean: -20.449885
episode 234.000000, reward total was -20.000000. running mean: -20.445386
episode 235.000000, reward total was -21.000000. running mean: -20.450933
episode 236.000000, reward total was -20.000000. running mean: -20.446423
episode 237.000000, reward total was -19.000000. running mean: -20.431959
episode 238.000000, reward total was -

episode 336.000000, reward total was -19.000000. running mean: -20.468822
episode 337.000000, reward total was -21.000000. running mean: -20.474134
episode 338.000000, reward total was -20.000000. running mean: -20.469392
episode 339.000000, reward total was -21.000000. running mean: -20.474698
episode 340.000000, reward total was -21.000000. running mean: -20.479951
episode 341.000000, reward total was -20.000000. running mean: -20.475152
episode 342.000000, reward total was -20.000000. running mean: -20.470400
episode 343.000000, reward total was -21.000000. running mean: -20.475696
episode 344.000000, reward total was -21.000000. running mean: -20.480939
episode 345.000000, reward total was -21.000000. running mean: -20.486130
episode 346.000000, reward total was -20.000000. running mean: -20.481269
episode 347.000000, reward total was -21.000000. running mean: -20.486456
episode 348.000000, reward total was -20.000000. running mean: -20.481591
episode 349.000000, reward total was -

...

episode 6502.000000, reward total was -17.000000. running mean: -19.074223
episode 6503.000000, reward total was -21.000000. running mean: -19.093481
episode 6504.000000, reward total was -21.000000. running mean: -19.112546
episode 6505.000000, reward total was -17.000000. running mean: -19.091421
episode 6506.000000, reward total was -20.000000. running mean: -19.100506
episode 6507.000000, reward total was -19.000000. running mean: -19.099501
episode 6508.000000, reward total was -18.000000. running mean: -19.088506
episode 6509.000000, reward total was -20.000000. running mean: -19.097621
episode 6510.000000, reward total was -21.000000. running mean: -19.116645
episode 6511.000000, reward total was -18.000000. running mean: -19.105479
episode 6512.000000, reward total was -18.000000. running mean: -19.094424
episode 6513.000000, reward total was -21.000000. running mean: -19.113480
episode 6514.000000, reward total was -19.000000. running mean: -19.112345
episode 6515.000000, rewa

episode 6612.000000, reward total was -19.000000. running mean: -19.265953
episode 6613.000000, reward total was -19.000000. running mean: -19.263294
episode 6614.000000, reward total was -20.000000. running mean: -19.270661
episode 6615.000000, reward total was -18.000000. running mean: -19.257954
episode 6616.000000, reward total was -18.000000. running mean: -19.245375
episode 6617.000000, reward total was -19.000000. running mean: -19.242921
episode 6618.000000, reward total was -20.000000. running mean: -19.250492
episode 6619.000000, reward total was -15.000000. running mean: -19.207987
episode 6620.000000, reward total was -19.000000. running mean: -19.205907
episode 6621.000000, reward total was -20.000000. running mean: -19.213848
episode 6622.000000, reward total was -20.000000. running mean: -19.221709
episode 6623.000000, reward total was -19.000000. running mean: -19.219492
episode 6624.000000, reward total was -20.000000. running mean: -19.227297
episode 6625.000000, rewa

episode 6722.000000, reward total was -21.000000. running mean: -19.304957
episode 6723.000000, reward total was -21.000000. running mean: -19.321907
episode 6724.000000, reward total was -19.000000. running mean: -19.318688
episode 6725.000000, reward total was -20.000000. running mean: -19.325501
episode 6726.000000, reward total was -21.000000. running mean: -19.342246
episode 6727.000000, reward total was -18.000000. running mean: -19.328824
episode 6728.000000, reward total was -21.000000. running mean: -19.345536
episode 6729.000000, reward total was -15.000000. running mean: -19.302080
episode 6730.000000, reward total was -19.000000. running mean: -19.299060
episode 6731.000000, reward total was -19.000000. running mean: -19.296069
episode 6732.000000, reward total was -18.000000. running mean: -19.283108
episode 6733.000000, reward total was -20.000000. running mean: -19.290277
episode 6734.000000, reward total was -20.000000. running mean: -19.297374
episode 6735.000000, rewa

episode 6832.000000, reward total was -20.000000. running mean: -19.157850
episode 6833.000000, reward total was -17.000000. running mean: -19.136271
episode 6834.000000, reward total was -21.000000. running mean: -19.154908
episode 6835.000000, reward total was -19.000000. running mean: -19.153359
episode 6836.000000, reward total was -19.000000. running mean: -19.151826
episode 6837.000000, reward total was -19.000000. running mean: -19.150307
episode 6838.000000, reward total was -19.000000. running mean: -19.148804
episode 6839.000000, reward total was -20.000000. running mean: -19.157316
episode 6840.000000, reward total was -19.000000. running mean: -19.155743
episode 6841.000000, reward total was -20.000000. running mean: -19.164186
episode 6842.000000, reward total was -20.000000. running mean: -19.172544
episode 6843.000000, reward total was -19.000000. running mean: -19.170818
episode 6844.000000, reward total was -18.000000. running mean: -19.159110
episode 6845.000000, rewa

episode 6942.000000, reward total was -20.000000. running mean: -18.985767
episode 6943.000000, reward total was -21.000000. running mean: -19.005909
episode 6944.000000, reward total was -17.000000. running mean: -18.985850
episode 6945.000000, reward total was -21.000000. running mean: -19.005992
episode 6946.000000, reward total was -20.000000. running mean: -19.015932
episode 6947.000000, reward total was -19.000000. running mean: -19.015773
episode 6948.000000, reward total was -21.000000. running mean: -19.035615
episode 6949.000000, reward total was -17.000000. running mean: -19.015259
episode 6950.000000, reward total was -20.000000. running mean: -19.025106
episode 6951.000000, reward total was -21.000000. running mean: -19.044855
episode 6952.000000, reward total was -19.000000. running mean: -19.044407
episode 6953.000000, reward total was -18.000000. running mean: -19.033963
episode 6954.000000, reward total was -20.000000. running mean: -19.043623
episode 6955.000000, rewa